## 4-2 データ処理の実習

### データの選択 − 気象データの読み込み

In [ ]:
import pandas as pd
wea = pd.read_csv('data/Weather_data.csv',
                  parse_dates=['日時'], index_col = '日時')
wea.head()

- pandas：データ分析を効率的に行うためのライブラリ
  - データの読み込みや統計量の表示、グラフ化、データ分析に関する作業を容易に行うことができる
- pd.read_csv(ファイル名, ･･･ )：CSVファイルの読み込み
  - 読み込み時に「日時」列を日付型およびインデックスに設定する

### データの選択 − 使用データ項目の抽出

In [ ]:
weax = wea[['気温', '湿度', '日照時間']]
weax = weax.replace({'--':0, '///':None}).astype('float64')
weax.head()

- weax：「気温」「湿度」「日照時間」を使用項目として抽出したデータ
- weax.replace：データの「--」を「0」、「///」を「None」へ変換する
  - 「--」：気象現象が観測されていないことを示す（値は0）
  - 「///」：データが記録できなかったことを示す（欠損値）

### クリーニング − 欠損値を含むレコード件数の確認

In [ ]:
print(len(weax))
weax.isnull().sum()

- 全レコード数と欠損値の件数を確認して処理方法を決める
  - このデータの場合、欠損値件数が極めて少ないので、欠損値を含むレコードは削除する

### クリーニング − 欠損値を含むレコードの削除

In [ ]:
weax = weax.dropna()
print(len(weax))
print(weax.isnull().sum())
weax.describe()

- weax.dropna()：欠損値を含むレコードを削除する
- len(weax)：全レコード数を確認
- weax.isnull().sum()：念のため項目毎の欠損値（null）の合計数を調べる
- weax.describe()：基本統計量（平均、標準偏差、最小、最大、四分位数）を確認

### クリーニング − 外れ値の確認（四分位範囲による判定）

In [ ]:
for s in ['気温', '湿度']:
    q1 = weax[s].describe()['25%'] # 第1四分位数（Q1）を求める
    q3 = weax[s].describe()['75%'] # 第3四分位数（Q3）を求める
    #四分位範囲（IQR）を求める
    iqr = q3 - q1
    # 外れ値の基準設定
    outl_min = q1 - 1.5 * iqr  
    outl_max = q3 + 1.5 * iqr
    # 外れ値の範囲確認と判定
    print('{:}: {:.1f} 〜 {:.1f}'.format(s, outl_min, outl_max))  
    outlier = weax[(weax[s] < outl_min) | (weax[s] > outl_max)]
outlier

- for s in ['気温', '湿度']：外れ値検出対象の選択
  - 日照時間は最小「0」、最大「1」であり、適切な範囲のため外れ値検出の対象外とする
- 湿度の外れ値が6件検出されるが、取り得る値と判断し、削除せずそのまま使用する

### データの構築 − 派生項目の追加

In [ ]:
weax['曜日'] = weax.index.day_name()
weax.head()

- 電力消費量は曜日によって変化と考え、曜日を派生項目として追加する
- weax['曜日'] = weax.index.day_name()
  - weaxのインデックス（日時）から曜日（day_name）を求めて「曜日」列に追加
  - 曜日は英語表記（Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday）

### データの統合 − 電力消費量データの読み込み

In [ ]:
elec = pd.read_csv('data/Electricity_data.csv',
                   parse_dates ={'日時': ['DATE', 'TIME']},
                   index_col = '日時')
elec.head()

- 電力消費量データを読み込んでデータ統合の準備をする
- 読み込み時に ['DATE', 'TIME'] 列を「日時」とし、日付型およびインデックスに設定する

### データの統合 − 気象データと電力消費量データの統合

In [ ]:
joined_data = weax.join(elec)
joined_data.head()

- 日時をキーに気象データと電力消費量データを結合する
- weax.join(elec)：インデックス（日時）をキーにして結合する方法

### フォーマット変換 − 文字データを数値へ変換

In [ ]:
target_data = pd.get_dummies(joined_data)
target_data.head()

- 曜日をフラグ化する（多値データのOne-Hotベクトル化）
- pd.get_dummies()：文字データを対象にフラグ化するpandasの関数

### フォーマット変換 − CSVファイル出力

In [ ]:
target_data.to_csv('data/weather-elec_data.csv')

- モデル作成用にCSVファイルへ出力する
  - target_data.to_csv()：データ（target_data）をCSVファイルへ出力する
- 出力したCSVファイルは、機械学習の実習で使用する